In [ ]:
# import all the needed module

from pandas import DataFrame, read_csv

import itertools
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
import seaborn as sb
from scipy.io import loadmat 
import pandas as pd
plt.style.use('seaborn-poster')
%matplotlib inline

In [ ]:
#islak data
w_aa_l = loadmat('./wet_30.mat')['sw_aw_30']
w_aa_l = w_aa_l[:  , 1:]
w_p_l = loadmat('./wet_30.mat')['sw_pp_30']
w_p_l = w_p_l[:  , 1:]
w_tv_l = loadmat('./wet_30.mat')['sw_tw_30']
w_tv_l = w_tv_l[:  , 1:]
w_vv_l = loadmat('./wet_30.mat')['sw_v_30']
w_vv_l = w_vv_l[:  ,  1:]
y = np.zeros((1646, 1))

In [ ]:
w_aa_l.shape

In [ ]:
dataset = np.stack((w_aa_l, w_p_l,w_tv_l,w_vv_l,y), axis=1)

In [ ]:
xx = loadmat('./slippery_30.mat')
xx

In [ ]:
#kaygan data
k_aa_l2 = loadmat('./slippery_30.mat')['sk_aw_30']
k_aa_l2 = k_aa_l2[:  ,  1:] #iki tane kolon var
p_l2 = loadmat('./slippery_30.mat')['sk_pp_30']
p_l2 = p_l2[:  ,  1:]
k_tv_l2 = loadmat('./slippery_30.mat')['sk_tw_30']
k_tv_l2 = k_tv_l2[: , 1:]
k_vv_l2 = loadmat('./slippery_30.mat')['sk_v_30']
k_vv_l2 = k_vv_l2[:  ,  1:]
y2 = np.ones((1445, 1))

In [ ]:
k_aa_l2.shape

In [ ]:
dataset_kaygan=np.stack((k_aa_l2, p_l2,k_tv_l2,k_vv_l2,y2), axis=1)

In [ ]:
#toplam data
new_30 = np.vstack((dataset, dataset_kaygan))

In [ ]:
nsamples, nx, ny = new_30.shape
d2_train_dataset = new_30.reshape((nsamples,nx*ny))

In [ ]:
#file = r'datasets/purchase_data.xls'
#purchase=pd.read_excel(file)
#purchase.drop('Unnamed: 0',axis=1, inplace=True)

#Assign predictors to a variable of ndarray (matrix) type
#array = purchase.values
X = d2_train_dataset[:,0:3] # features
y = d2_train_dataset[:,-1]

#Divide records in training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2, stratify=y)

scaler = StandardScaler()
# Fit only to the training data
scaler.fit(X_train)

# Now apply the transformations to the data:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
print('Number of samples in training set: %d, number of samples in test set: %d'%(len(y_train), len(y_test)))

In [ ]:
# Initialize ANN classifier
mlp = MLPClassifier(hidden_layer_sizes=(40,40,40), activation='logistic', max_iter = 4000)

# Train the classifier with the traning data
mlp.fit(X_train_scaled,y_train)

In [ ]:
# predict results from the test data
predicted = mlp.predict(X_test_scaled)

# plot the confusion matrix
cm = confusion_matrix(y_test,predicted)
print(classification_report(y_test, predicted))

fig, ax = plt.subplots(figsize=(5, 5))
ax.matshow(cm, cmap=plt.cm.Reds, alpha=0.3)
for i in range(cm.shape[0]):
     for j in range(cm.shape[1]):
         ax.text(x=j, y=i,
                s=cm[i, j], 
                va='center', ha='center')
plt.xlabel('Predicted Values', )
plt.ylabel('Actual Values')
plt.show()

In [ ]:
#A combination of under- and oversampling method using pipeline
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
over = SMOTE(sampling_strategy='minority')
under = RandomUnderSampler(sampling_strategy='majority')
steps = [('u', under), ('o', over), ('model', mlp)]
pipeline = Pipeline(steps=steps)

# import libraries for evaluation
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from numpy import mean

# evaluate pipeline
scores = cross_val_score(pipeline, X_train, y_train, scoring='roc_auc', cv=5, n_jobs=-1)
score = mean(scores)
print('ROC AUC score for the combined sampling method: %.3f' % score)